In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics,neighbors
from sklearn.preprocessing import MinMaxScaler

import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense ,Dropout,BatchNormalization
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import os
from tensorflow.keras.optimizers import SGD
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_train = pd.read_csv('../input/titanic/train.csv')
df_test = pd.read_csv('../input/titanic/test.csv')

In [ ]:
df_train.describe()

In [ ]:
df_train['Gender'] = 0
df_train['Gender'][(df_train['Sex']=='male')] = '0'
df_train['Gender'][(df_train['Sex']=='female')] = '1'
df_train=df_train.drop(['Sex'],axis=1)
df_train

In [ ]:
import seaborn as sns
corr = df_train.corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)

In [ ]:
df_train.dropna(subset=['Gender','Age','SibSp','Parch','Survived'], inplace = True) #Drop the columns with null value

In [ ]:
x_train=df_train[['Gender','Age','SibSp','Parch']] #Define input of train set
y_train = df_train[['Survived']] #Output of train set


In [ ]:
datatest1=pd.read_csv('../input/titanic/gender_submission.csv')
df_test = pd.read_csv('../input/titanic/test.csv')
df_test=pd.merge(df_test,datatest1,on='PassengerId',how='inner')
df_test

In [ ]:
df_test['Gender'] = 0
df_test['Gender'][(df_test['Sex']=='male')] = '0'
df_test['Gender'][(df_test['Sex']=='female')] = '1'
df_test=df_test.drop(['Sex'],axis=1)
df_test

In [ ]:
#df_test.dropna(subset=['Gender','Age','SibSp','Parch','Survived'], inplace = True)

In [ ]:
x_test=df_test[['Gender','Age','SibSp','Parch']]#Define input of test set
y_test = df_test[['Survived']] #Output of test set

In [ ]:
y_test

In [ ]:
#Scale the date before training

y_train=np.reshape(y_train, (-1,1))
y_test=np.reshape(y_test, (-1,1))
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
print(scaler_x.fit(x_train))
xtrain_scale=scaler_x.transform(x_train)
print(scaler_x.fit(x_test))
xtest_scale=scaler_x.transform(x_test)
print(scaler_y.fit(y_train))
ytrain_scale=scaler_y.transform(y_train)
print(scaler_y.fit(y_test))
ytest_scale=scaler_y.transform(y_test)

In [ ]:
xtest_scale

In [ ]:
model = Sequential()
model.add(Dense(30, input_dim=4, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(20,  activation='relu', kernel_initializer='he_normal'))
model.add(Dense(20, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(10, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(2, activation='softmax'))


In [ ]:

# compile the keras model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',metrics= ['accuracy'])

In [ ]:
#Fit the model to the train set
model.fit(xtrain_scale, ytrain_scale, epochs=70, batch_size=16, verbose=2,validation_split = 0.1)

In [ ]:
model.evaluate(xtest_scale, ytest_scale) # Evaluate the model on the unseen test data

In [ ]:
y_test

In [ ]:
x_test

In [ ]:
pred = model.predict(xtest_scale)

In [ ]:
df_test

In [ ]:
Final_output = np.argmax(pred, axis = 1)

In [ ]:
df_test['Survived'] = Final_output

In [ ]:
df_test_final = df_test[['PassengerId','Survived' ]]

In [ ]:
df_test_final

In [ ]:
df_test_final.to_csv('/kaggle/working/submission.csv', index=False)